In [2]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2021,21)

In [3]:
def deterministic_die(sides=100):
    value = 1
    while True:
        yield value
        value += 1
        if value > sides:
            value = 1


In [4]:
input_text = """Player 1 starting position: 4
Player 2 starting position: 8"""

In [5]:
import re
digit_pattern = re.compile(r'(\d+)')
def parse_data(lines):
    player_data = []
    for line in lines:
        player_num, position = digit_pattern.findall(line)
        player_data.append({
            "position": int(position),
            "name": f"Player {player_num}",
            "score": 0
        })
    return player_data


die = deterministic_die()
players = parse_data(input_text.splitlines())
# print(players)
roll_count = 0
while not any((player['score'] >=1000 for player in players)):
    for player in players:
        rolls = (next(die), next(die), next(die))
        roll_count += 3
        player['position'] = (player['position'] + sum(rolls)) % 10
        roll_str = '+'.join((str(roll) for roll in rolls))
        if player['position'] == 0:
            player['score'] += 10
            # print(f"{player['name']} rolls {roll_str} and moves to space 10 for a total score of {player['score']}.")
        else:
            player['score'] += player['position']
            # print(f"{player['name']} rolls {roll_str} and moves to space {player['position']} for a total score of {player['score']}.")
        # break here in case player 1 finishes first, so player 2 doesn't move again
        if player['score'] >= 1000:
            break
# print(f"{roll_count} rolls")
losing_score = min((player['score'] for player in players))
# print(f"{losing_score} * {roll_count} = {losing_score * roll_count}")
print(losing_score * roll_count)

    
            







739785


In [6]:
from itertools import combinations_with_replacement
from functools import lru_cache
from heapq import heappop, heappush

roll_combos = tuple(combinations_with_replacement((1,2,3), 3))

@lru_cache(maxsize=None)
def next_position(pos, rolls):
    new_pos =  (pos + sum(rolls)) % 10
    if new_pos == 0:
        return 10
    return new_pos

@lru_cache(maxsize=None)
def do_turn(turn, pos, scores, rolls):
    # print(f"do_turn({turn}, {pos}, {scores}, {rolls})")
    pos = list(pos)
    pos[turn] = next_position(pos[turn], rolls)
    scores = list(scores)
    # print(f"new position: {pos[turn]}")
    scores[turn] += pos[turn]
    # break here in case player 1 finishes first, so player 2 doesn't move again
    if scores[turn] >= 21:
        # single game winner, return (0, 1) or (1, 0)
        # print(f"returning True, {tuple(scores)}")
        return True, tuple(scores)
    # print(f"returning False, {tuple(scores)}")
    return False, tuple(scores)


def play_game(players, goal):
    turn = 0
    pos = [player['position'] for player in players]
    scores = (0, 0)
    heap = []
    wins = [0, 0]

    heappush(heap, (0, 0, tuple(pos), (0,0), ()))
    while heap:
        if len(heap) > 1000:
            print(f"breaking loop")
            break
        high_score, turn, pos, scores, rolls = heappop(heap)
        if high_score < 5:
             print(f"heap: {len(heap)}, high_score: {high_score}")
             print(wins)
        # print(high_score, turn, pos, scores, rolls)
        # new turn, process roll combos
        if len(rolls) == 0:
            # print(f"{turn} play roll combos")
            high_score = max(scores)
            for combo in roll_combos:
                heappush(heap, (21 - high_score, turn, pos, scores, combo))
            continue

        if turn == 0:
            next_turn = 1
        else:
            next_turn = 0
        # print(f"do_turn({turn}, {pos}, {scores}, {rolls})")
        # turn with rolls, lets move the player
        win, scores = do_turn(turn, pos, scores, rolls)
        # print(f"win: {win}, scores: {scores}")
        if win:
            # print(f"Winner! {turn}")
            wins[turn] += 1
            continue
        high_score = max(scores)
        # print(f"Queue next player: {(21 - high_score, next_turn, tuple(pos), tuple(scores), ())} ")
        # not a winner, play next turn
        heappush(heap,(21 - high_score, next_turn, tuple(pos), tuple(scores), ()))
    return wins


goal = 21
players = parse_data(input_text.splitlines())
# print(players)
# wins = play_turn(0,tuple((player['position'] for player in players)), (0,0))
# print(wins)
# wins = play_game(players, goal)
# print(wins)




In [7]:
import functools
import itertools


data = input_text.splitlines()

# part 1
p1, p2 = int(data[0][-1]), int(data[1][-1])
s1 = s2 = 0
die = rolls = 0
while True:
    die = die % 100 + 1
    if die % 2:  # player 1
        p1 += sum([die, die + 1, die + 2])
        s1 += p1 % 10 if p1 % 10 else 10
    else:  # player 2
        p2 += sum([die, die + 1, die + 2])
        s2 += p2 % 10 if p2 % 10 else 10
    die += 2
    rolls += 3
    if s1 >= 1000 or s2 >= 1000:
        break
print(f"Part 1: {min(s1, s2) * rolls}")


# part 2
@functools.lru_cache(maxsize=None)
def play_out(p1, s1, p2, s2):
    w1 = w2 = 0
    for m1, m2, m3 in itertools.product((1, 2, 3), (1, 2, 3), (1, 2, 3)):
        p1_copy = (p1 + m1 + m2 + m3) % 10 if (p1 + m1 + m2 + m3) % 10 else 10
        s1_copy = s1 + p1_copy
        if s1_copy >= 21:
            w1 += 1
        else:
            w2_copy, w1_copy = play_out(p2, s2, p1_copy, s1_copy)
            w1 += w1_copy
            w2 += w2_copy
    return w1, w2


p1, p2 = int(data[0][-1]), int(data[1][-1])
s1 = s2 = 0
print(f"Part 2: {max(play_out(p1, s1, p2, s2))}")

Part 1: 739785
Part 2: 444356092776315
